# Exercises

## Data Acquisition

These exercises should go in a notebook or script named `wrangle`. Add, commit, and push your changes.

This exercises uses the `case.csv`, `dept.csv`, and `source.csv` files from the san antonio 311 call dataset.



In [1]:
import pyspark
from pyspark.sql.functions import *

spark = pyspark.sql.SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/16 18:21:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Read the case, department, and source data into their own spark dataframes.



In [2]:
df = case = spark.read.csv('case.csv', header=True, inferSchema=True)


In [3]:
depts = spark.read.csv('dept.csv', header=True, inferSchema=True)
sources = spark.read.csv('source.csv', header=True, inferSchema=True)


In [4]:
df = df.join(depts, 'dept_division', 'left').join(sources, 'source_id', 'left')


In [5]:
df.show(5)

+---------+----------------+----------+----------------+----------------+------------+---------+-------------------+-----------+--------------------+-----------+-----------+--------------------+----------------+--------------------+----------------------+-------------------+---------------+
|source_id|   dept_division|   case_id|case_opened_date|case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|service_request_type|   SLA_days|case_status|     request_address|council_district|           dept_name|standardized_dept_name|dept_subject_to_SLA|source_username|
+---------+----------------+----------+----------------+----------------+------------+---------+-------------------+-----------+--------------------+-----------+-----------+--------------------+----------------+--------------------+----------------------+-------------------+---------------+
| svcCRMLS|Field Operations|1014127332|     1/1/18 0:42|    1/1/18 12:29|9/26/20 0:42|       NO| -998.5087616000001|        

Let's see how writing to the local disk works in spark:

Write the code necessary to store the source data in both `csv` and `json` format, store these as `sources_csv` and `sources_json`



In [12]:
df.write.csv('sources_csv.csv')


Inspect your folder structure. What do you notice?



Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.



In [6]:
df.dtypes

[('source_id', 'string'),
 ('dept_division', 'string'),
 ('case_id', 'int'),
 ('case_opened_date', 'string'),
 ('case_closed_date', 'string'),
 ('SLA_due_date', 'string'),
 ('case_late', 'string'),
 ('num_days_late', 'double'),
 ('case_closed', 'string'),
 ('service_request_type', 'string'),
 ('SLA_days', 'double'),
 ('case_status', 'string'),
 ('request_address', 'string'),
 ('council_district', 'int'),
 ('dept_name', 'string'),
 ('standardized_dept_name', 'string'),
 ('dept_subject_to_SLA', 'string'),
 ('source_username', 'string')]

- Make `council_district` string
- Make `case_late`and `case_closed` boolean
- Rename due date column
- Convert all dates to datetime from string

In [7]:
df = df.withColumnRenamed('SLA_due_date', 'case_due_date')


In [8]:
df = (
    df.withColumn('case_late', col('case_late') == 'YES')
    .withColumn('case_closed', col('case_closed') == 'YES')
)
df = df.withColumn('council_district', format_string('%04d', col('council_district')))
df = (
    df.withColumn('case_opened_date', to_timestamp(col('case_opened_date'), 'M/d/yy H:mm'))
    .withColumn('case_closed_date', to_timestamp(col('case_closed_date'), 'M/d/yy H:mm'))
    .withColumn('case_due_date', to_timestamp(col('case_due_date'), 'M/d/yy H:mm'))
)

In [9]:
df.show(5)

+---------+----------------+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+--------------------+-----------+-----------+--------------------+----------------+--------------------+----------------------+-------------------+---------------+
|source_id|   dept_division|   case_id|   case_opened_date|   case_closed_date|      case_due_date|case_late|      num_days_late|case_closed|service_request_type|   SLA_days|case_status|     request_address|council_district|           dept_name|standardized_dept_name|dept_subject_to_SLA|source_username|
+---------+----------------+----------+-------------------+-------------------+-------------------+---------+-------------------+-----------+--------------------+-----------+-----------+--------------------+----------------+--------------------+----------------------+-------------------+---------------+
| svcCRMLS|Field Operations|1014127332|2018-01-01 00:42:00|2018-01-01 12:29:00|2020-0

How old is the latest (in terms of days past SLA) currently open issue? 

How long has the oldest (in terms of days since opened) currently opened issue been open?


How many Stray Animal cases are there?


How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?



Convert the council_district column to a string column.



Extract the year from the case_closed_date column.


Convert num_days_late from days to hours in new columns num_hours_late.



Join the case data with the source and department data.



Are there any cases that do not have a request source?



What are the top 10 service request types in terms of number of requests?



What are the top 10 service request types in terms of average days late?


Does number of days late depend on department?


How do number of days late depend on department and request type?


You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.